In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jax import vmap
from jax import jit, lax
import matplotlib.pyplot as plt
import cmocean as cmo
import importlib

In [2]:
from jax.config import config
config.update("jax_enable_x64", True)

In [33]:
## import modules
import preconditioner as precond
import conjugate_gradient as cg
import pivoted_cholesky as pc
import pivoted_cholesky_ref as pc_ref # to use this script we need "torch", please comment out if not needed.
def reload():
    importlib.reload(precond)
    importlib.reload(cg)
    importlib.reload(pc)
    importlib.reload(pc_ref)
reload()

In [4]:
import gpytorch
import torch
import linear_operator
from linear_operator.operators import (
    AddedDiagLinearOperator,
    DiagLinearOperator,
    LinearOperator,
    DenseLinearOperator,
)

In [5]:
def generate_K(N, seed=0, noise=1e-06):
    """
    generate positive definite symmetric matrix
    """
    K = jax.random.normal(jax.random.PRNGKey(seed), (N, N))
    # K = K @ K.T + 30* jnp.eye(N) + noise*jnp.eye(N)
    # K = jnp.dot(K, K.T) + noise*jnp.eye(N)
    # K = jnp.dot(K, K.T) / N
    K = jnp.dot(K, K.T)/N
    # K += (noise+30)*jnp.eye(N) ## ??
    K += (5)*jnp.eye(N)
    K += (noise)*jnp.eye(N)
    if not is_positive_definite(K):
        raise Exception("K is not positive definite !")
    return K

In [6]:
def is_positive_definite(matrix):
    # 行列の固有値を計算
    eigenvalues = np.linalg.eigvals(matrix)

    # 全ての固有値が正であるかをチェック
    if np.all(eigenvalues > 0):
        return True
    else:
        return False

In [7]:
def rel_error(true, pred):
    nonzero_index = jnp.where(true != 0.)
    true = true[nonzero_index]
    pred = pred[nonzero_index]
    return jnp.mean(jnp.abs((true-pred)/true))

## 4. preconditioned batched conjugate gradient

In [27]:
N = 3000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [41]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [29]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)

In [34]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)

In [34]:
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1)
print(rel_error(Kinvy_linalg, Kinvy))

0.14856566444817215


In [43]:
linear_operator.settings.cg_tolerance._set_value(1)
Kinvy_torch = added_diag.solve(rhs_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
linear_operator.settings.cg_tolerance._set_value(1)

0.1485654807791762


### tol=1e-04

In [44]:
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1e-04)
print(rel_error(Kinvy_linalg, Kinvy))

0.0004721970976828854


In [46]:
linear_operator.settings.cg_tolerance._set_value(1e-04)
Kinvy_torch = added_diag.solve(rhs_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
# linear_operator.settings.cg_tolerance._set_value(1)

0.00047221962753508294


In [50]:
N = 7000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [51]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [52]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)

In [53]:
%%timeit
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1e-04)

258 ms ± 1.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%%timeit
Kinvy_torch = added_diag.solve(rhs_torch)

1.52 s ± 4.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
%%timeit
Kinvy_linalg = jnp.linalg.solve(K, rhs)

751 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
print(rel_error(Kinvy_linalg, Kinvy))
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

0.0004721970976828854
0.00047221962753508294


In [57]:
%%timeit
Kinvy = cg.mpcg_bbmm(K, rhs, preconditioner=None, print_process=False, tolerance=1e-04)

156 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### やはりpreconditionしない方がはやいのが現状

## chceking trace term

### with preconditioner

In [76]:
N = 3000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [77]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [78]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)

In [79]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1)
print(rel_error(Kinvy_linalg, Kinvy))
linear_operator.settings.cg_tolerance._set_value(1)
Kinvy_torch = added_diag.solve(rhs_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
linear_operator.settings.cg_tolerance._set_value(1)

0.14856566444817215
0.1485654807791762


In [57]:
%%timeit
Kinvy = cg.mpcg_bbmm(K, rhs, preconditioner=None, print_process=False, tolerance=1e-04)

156 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [80]:
dKdtheta = jax.random.normal(jax.random.PRNGKey(1), (N, N))+5*jnp.eye(N)

In [86]:
trace = jnp.einsum("ij, ij ->", Kinvy[:, :n_tridiag], jnp.einsum("ij, jk->ik", dKdtheta, zs))/n_tridiag

In [87]:
Kinvy_linalg = jnp.linalg.inv(K)
trace_linalg = jnp.sum(jnp.diag(jnp.matmul(Kinvy_linalg, dKdtheta)))

In [90]:
trace, trace_linalg

(DeviceArray(2674.62575194, dtype=float64),
 DeviceArray(2587.2629889, dtype=float64))

In [89]:
(trace_linalg-trace)/trace_linalg

DeviceArray(-0.03376648, dtype=float64)

### without preconditioner

In [100]:
Kinvy = cg.mpcg_bbmm(K, rhs, preconditioner=None, print_process=False, tolerance=1)
print(rel_error(Kinvy_linalg, Kinvy))

3448.798220099444


In [101]:
dKdtheta = jax.random.normal(jax.random.PRNGKey(1), (N, N))+5*jnp.eye(N)

In [106]:
trace = jnp.einsum("ij, ij ->", Kinvy[:, :n_tridiag], jnp.einsum("ij, jk->ik", dKdtheta, zs))/n_tridiag

In [107]:
Kinvy_linalg = jnp.linalg.inv(K)
trace_linalg = jnp.sum(jnp.diag(jnp.matmul(Kinvy_linalg, dKdtheta)))

In [108]:
trace, trace_linalg

(DeviceArray(2671.43610977, dtype=float64),
 DeviceArray(2587.2629889, dtype=float64))

In [109]:
(trace_linalg-trace)/trace_linalg

DeviceArray(-0.03253365, dtype=float64)